In [39]:
import pandas as pd
import numpy as np

# 당뇨 데이터 불러오기
df = pd.read_excel('diabetes.xlsx', engine="openpyxl", index_col = 0)
df.head()

,임신,글루코스(탄수화물 화합물),혈압,피부두께,인슐린,BMI,가족력,나이,당뇨여부
ID,,,,,,,,,
1,6,148,72,35,0,33.6,0.627,50,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
6,2,197,70,45,543,30.5,0.158,53,1
8,5,166,72,19,175,25.8,0.587,51,1


In [14]:
# 데이터 확인
print(df.shape)
print()

# 타겟 변수가 편향 되어 있음
print(df.당뇨여부.value_counts())

(376, 9)

0    249
1    127
Name: 당뇨여부, dtype: int64


In [51]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

# 타겟 변수가 편향 되어 있을때는 '정확도' 보다는 정밀도'를 사용하는게 좋다.
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,:8], df.iloc[:,8], test_size=0.3, random_state = 0)

# 파라미터를 디폴트 값으로 설정
Ada_model = AdaBoostClassifier(n_estimators=50,
                         learning_rate=1,
                        random_state = 0)

# fitting
model = Ada_model.fit(X_train, y_train)

# predict
Ada_pred = model.predict(X_test)

In [52]:
## 정밀도 계산하는 함수

def true_positive(y_true, y_pred):
    """
    TP 계산하는 함수
    param y_true : 실제 값의 목록
    param y_pred : 예측 값의 목록
    return : TP의 개수
    """
    # 초기화
    tp = 0
    for yt, yp in zip(y_true, y_pred):
        if yt == 1 and yp == 1:
            tp += 1
    return tp

def true_negative(y_true, y_pred):
    """
    TN을 계산하는 함수
    param y_true : 실제 값의 목록
    param y_pred : 예측 값의 목록
    return : TN의 개수
    """
    # 초기화
    tn = 0
    for yt, yp in zip(y_true, y_pred):
        if yt == 0 and yp == 0:
            tn += 1
    return tn

def false_positive(y_true, y_pred):
    """
    FP을 계산하는 함수
    param y_true : 실제 값의 목록
    param y_pred : 예측 값의 목록
    return : FP의 개수
    """
    # 초기화
    fp = 0
    for yt, yp in zip(y_true, y_pred):
        if yt == 0 and yp == 1:
            fp += 1
    return fp

def false_negative(y_true, y_pred):
    """
    FN을 계산하는 함수
    param y_true : 실제 값의 목록
    param y_pred : 예측 값의 목록
    return : FN의 개수
    """
    # 초기화
    fn = 0
    for yt, yp in zip(y_true, y_pred):
        if yt == 1 and yp == 0:
            fn += 1
    return fn


## 정밀도 구현 코드
def precision(y_true, y_pred):
    tp = true_positive(y_true, y_pred)
    fp = false_positive(y_true, y_pred)
    precision = tp / (tp+fp)
    return precision

# 정밀도 : postive로 예측한 수 중 실제 일치
precision(np.array(y_test), Ada_pred)

0.6888888888888889

In [53]:
# 사이킷런을 활용하기
print('accuracy', metrics.accuracy_score(np.array(y_test), Ada_pred) )
print()
print('precision', metrics.precision_score(np.array(y_test), Ada_pred) )
print()
print('recall', metrics.recall_score(np.array(y_test), Ada_pred) )
print()
print('f1', metrics.f1_score(np.array(y_test), Ada_pred) )

accuracy 0.7964601769911505

precision 0.6888888888888889

recall 0.775

f1 0.7294117647058822


In [94]:
# 파라미터 수정
Ada_model = AdaBoostClassifier(n_estimators=50,
                         learning_rate=0.1,
                        random_state = 0)

# fitting
model = Ada_model.fit(X_train, y_train)

# predict
Ada_pred = model.predict(X_test)

In [95]:
# 사이킷런을 활용하기
print('accuracy', metrics.accuracy_score(np.array(y_test), Ada_pred) )
print()
print('precision', metrics.precision_score(np.array(y_test), Ada_pred) )
print()
print('recall', metrics.recall_score(np.array(y_test), Ada_pred) )
print()
print('f1', metrics.f1_score(np.array(y_test), Ada_pred) )

accuracy 0.7699115044247787

precision 0.71875

recall 0.575

f1 0.6388888888888888


- 정밀도 기준으로는 learning_rate = 0.1로 설정하는게 가장 좋은 성능을 낸다.
- Adaboost는 파라미터가 많지 않아 데이터가 크지 않다면 파라미터에 따른 성능 확인이 어렵지 않을거라 생각한다.